In [8]:
def create_locations_map(gdf):
    """
    Create Folium map with location markers using TEEHR visualization patterns.
    
    Args:
        gdf: GeoDataFrame with location data and geometries
        
    Returns:
        folium.Map: Interactive map centered on data extent
    """
    if gdf is None or len(gdf) == 0:
        # Default center if no data (geographic center of CONUS for TEEHR)
        center_lat, center_lon = 39.8283, -98.5795
    else:
        # Calculate center from GeoDataFrame bounds for optimal view
        # This follows TEEHR patterns for efficient geospatial processing
        bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
        center_lon = (bounds[0] + bounds[2]) / 2  # (minx + maxx) / 2
        center_lat = (bounds[1] + bounds[3]) / 2  # (miny + maxy) / 2
    
    # Create base map with multiple tile layers for better visualization
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=4,
        tiles=None  # We'll add custom layers
    )

    # Add multiple tile layers following TEEHR dashboard patterns
    folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m)
    # folium.TileLayer('CartoDB positron', name='CartoDB Light').add_to(m)
    # folium.TileLayer('CartoDB dark_matter', name='CartoDB Dark').add_to(m)
    
    # Add markers for each location with data quality color coding
    for idx, row in gdf.iterrows():
        geom = row.geometry
        if geom and not geom.is_empty:
            # Color code by data quality bucket following TEEHR patterns
            # color_map = {
            #     'high': 'green',    # High quality: >10k records
            #     'medium': 'orange', # Medium quality: 1k-10k records
            #     'low': 'red'        # Low quality: <1k records
            # }
            # color = color_map.get(getattr(row, 'record_count_bucket', 'low'), 'blue')
            color = "blue"
            
            # Create rich popup with TEEHR metadata
            popup_content = f"""
            <div style='min-width: 200px'>
                <p>{row["name"]}</br>{row["primary_location_id"]}</p>
                <p><b>Records:</b> {int(row["count"]):.0f}</p>
                <p><b>Coordinates:</b> {geom.y:.4f}°, {geom.x:.4f}°</p>
                <button onclick="document.querySelector('select[name=\\'📍 Select Location\\']').value='{row['primary_location_id']}'; document.querySelector('select[name=\\'📍 Select Location\\']').dispatchEvent(new Event('change'));">
                    View Time Series
                </button>
            </div>
            """
            
            folium.CircleMarker(
                location=[geom.y, geom.x],
                popup=folium.Popup(popup_content, max_width=300),
                tooltip=f"{row["primary_location_id"]}:{row["name"]}",
                radius=8,
                color='white',
                weight=2,
                fillColor=color,
                fillOpacity=0.8
            ).add_to(m)
    
    # Add layer control for tile selection
    folium.LayerControl().add_to(m)
    
    return m

In [9]:
def create_timeseries_chart(primary_df, secondary_df, location_id: str):
    """
    Create plotly time series chart with multiple configurations as traces.
    
    Args:
        df: DataFrame with time series data
        location_id: Location identifier for chart title
        start_date: Start date for chart title
        end_date: End date for chart title
        
    Returns:
        plotly.graph_objects.Figure: Interactive time series chart
    """
    fig = go.Figure()
    
    # Get unique configurations
    configurations = primary_df['configuration_name'].unique()
    
    # Color palette for different configurations
    colors = px.colors.qualitative.Set3
    
    for i, config in enumerate(configurations):
        config_df = primary_df[primary_df['configuration_name'] == config]
        
        fig.add_trace(go.Scatter(
            x=config_df['value_time'],
            y=config_df['value'],
            mode='lines',
            name=config,
            line=dict(color="black"),
            hovertemplate='<b>%{fullData.name}</b><br>' +
                         'Time: %{x}<br>' +
                         'Flow: %{y:.2f} cfs<br>' +
                         '<extra></extra>'
        ))

    # Get unique configurations
    configurations = secondary_df['configuration_name'].unique()
    
    for j, config in enumerate(configurations):
        config_df = secondary_df[secondary_df['configuration_name'] == config]
        
        fig.add_trace(go.Scatter(
            x=config_df['value_time'],
            y=config_df['value'],
            mode='lines',
            name=config,
            line=dict(color="red"),
            hovertemplate='<b>%{fullData.name}</b><br>' +
                         'Time: %{x}<br>' +
                         'Flow: %{y:.2f} cfs<br>' +
                         '<extra></extra>'
        ))
        
    fig.update_layout(
        title=f'Hourly Streamflow for {location_id}', #<br><sub>{start_date} to {end_date}</sub>',
        xaxis_title='Date/Time',
        yaxis_title='Streamflow (cfs)',
        hovermode='x unified',
        showlegend=True,
        height=400,
        margin=dict(l=40, r=40, t=60, b=40)
    )
    
    return fig

In [10]:
class TEEHRDashboard:
    """
    Main dashboard class following TEEHR architecture patterns.
    
    Implements cloud-native hydrologic evaluation dashboard with:
    - ECS Trino integration for scalable data access
    - Interactive geospatial visualization
    - Time series analysis and comparison
    - Data quality assessment and visualization
    """
    
    def __init__(self):
        """Initialize dashboard components and state."""
        self.locations_gdf = None
        self.selected_location_id = None
        self.selected_location_data = None
        
        # Initialize Panel components
        self.setup_components()
        self.load_metrics_data()
    
    def setup_components(self):
        """Setup Panel dashboard components following TEEHR UI patterns."""
        
        # Header and configuration
        self.title = pn.pane.Markdown("""
        # 🗺️ TEEHR Simulation Evaluation Dashboard
        """)
        
        # Configuration panel
        self.config_info = pn.pane.Markdown(f"""
        ### 🔧 Database Connection Information
        ```
        Host: {TRINO_HOST}
        Port: {TRINO_PORT}
        User: {TRINO_USER}
        Catalog: {TRINO_CATALOG}
        Schema: {TRINO_SCHEMA}
        ```
        """)
        
        # Status indicator
        self.status_pane = pn.pane.Markdown("🔄 **Loading connection status...**")
        
        # Data refresh button
        self.refresh_button = pn.widgets.Button(
            name="🔄 Refresh Data", 
            button_type="primary",
            width=150
        )
        self.refresh_button.on_click(self.refresh_data)
        
        # Map component
        self.map_pane = pn.pane.HTML("Loading map...", height=600)
        
        # Summary statistics
        self.stats_pane = pn.pane.Markdown("Loading statistics...")
        
        # Location selection
        self.location_selector = pn.widgets.Select(
            name="📍 Select Location",
            options=[],
            width=300
        )
        self.location_selector.param.watch(self.on_location_change, 'value')
        
        # Date range selectors
        # today = date.today()
        # default_start = today - timedelta(days=30)
        
        # self.start_date = pn.widgets.DatePicker(
        #     name="📅 Start Date",
        #     value=default_start,
        #     end=today,
        #     width=150
        # )
        
        # self.end_date = pn.widgets.DatePicker(
        #     name="📅 End Date", 
        #     value=today,
        #     start=default_start,
        #     end=today,
        #     width=150
        # )
        
        # Time series chart
        self.chart_pane = pn.pane.Plotly(height=400)
        
        # Location details
        self.location_details = pn.pane.Markdown("Select a location to view details")
        
        # Data table
        self.data_table = pn.widgets.Tabulator(pagination='remote', page_size=20, height=300)
    
    def load_metrics_data(self):
        """Load locations data from Trino following TEEHR patterns."""
        self.status_pane.object = "🔄 **Loading data from ECS Trino...**"
        
        # Test connection first
        conn, conn_error = get_trino_connection()
        if conn_error:
            self.status_pane.object = f"❌ **Connection Failed**: {conn_error}"
            return
        
        # Load locations data
        gdf, error = get_locations_data()
        if error:
            self.status_pane.object = f"❌ **Data Loading Failed**: {error}"
            return
        
        self.locations_gdf = gdf
        self.update_components()
        self.status_pane.object = f"✅ **Connected**: Loaded {len(gdf)} locations"
    
    def update_components(self):
        """Update dashboard components with loaded data."""
        if self.locations_gdf is None:
            return
        
        # Update summary statistics
        gdf = self.locations_gdf
        bounds = gdf.total_bounds
        lat_range = bounds[3] - bounds[1]
        lon_range = bounds[2] - bounds[0]
        
        self.stats_pane.object = f"""
        ### 📊 Data Summary
        - **Total Locations**: {len(gdf):,}
        - **Valid Geometries**: {len(gdf[~gdf.geometry.is_empty]):,}
        - **Geographic Span**: {lat_range:.1f}° × {lon_range:.1f}°
        """
        
        # Update location selector
        location_options = [None, *list(gdf["primary_location_id"])]
        self.location_selector.options = location_options
        
        # Update map
        locations_map = create_locations_map(gdf)
        self.map_pane.object = locations_map._repr_html_()
        
        # Update data table
        display_df = pd.DataFrame(gdf)
        display_df = display_df.drop(columns=["geometry"]).copy()
        self.data_table.value = display_df
    
    def refresh_data(self, event=None):
        """Refresh data from Trino."""
        self.load_metrics_data()
    
    def on_location_change(self, event):
        """Handle location selection change."""
        if not event.new:
            return
        
        self.selected_location_id = event.new
        filtered = self.locations_gdf[self.locations_gdf.primary_location_id == event.new]
        if filtered.empty:
            self.location_details.object = f"⚠️ No data found for location: {event.new}"
            return
        selected_row = filtered.iloc[0]
        
        # Update location details
        geom = selected_row.geometry
        self.location_details.object = f"""
        ### 📍 Selected Location: {selected_row.primary_location_id}
        - **Name**: {selected_row.name}
        - **Coordinates**: {geom.y:.4f}°N, {geom.x:.4f}°W
        - **Records**: {selected_row.count}
        """

        # Set Selector widget value
        # self.location_selector.value = selected_row.primary_location_id
        
        # Load time series data
        self.load_timeseries()
    
    def load_timeseries(self):
        """Load time series data for selected location."""
        if not self.selected_location_id:
            return
        
        # Get date range
        # start_date = self.start_date.value
        # end_date = self.end_date.value
        
        # Load data
        primary_df, error = get_primary_timeseries(self.selected_location_id)
        
        if error:
            self.chart_pane.object = go.Figure().add_annotation(
                text=f"No time series data available: {error}",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            return

        secondary_df, error = get_secondary_timeseries(self.selected_location_id)

        if error:
            self.chart_pane.object = go.Figure().add_annotation(
                text=f"No time series data available: {error}",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            return
            
        # Create chart
        chart = create_timeseries_chart(primary_df, secondary_df, self.selected_location_id)
        self.chart_pane.object = chart
    
    def create_layout(self):
        """Create dashboard layout following TEEHR UI patterns."""
        
        # Sidebar with configuration and controls
        sidebar = pn.Column(
            self.config_info,
            self.status_pane,
            self.refresh_button,
            "---",
            self.stats_pane,
            "---",
            self.location_selector,
            self.location_details,
            # pn.Row(self.start_date, self.end_date),
            width=350,
            scroll=True
        )
        
        # Main content area
        main_content = pn.Column(
            self.title,
            # pn.pane.Markdown("🚀 **Using ECS-hosted Trino** to query TEEHR locations from Iceberg warehouse"),
            "---",
            pn.pane.Markdown("### 📍 Interactive Location Map"),
            self.map_pane,
            "---",
            pn.pane.Markdown("### 📋 Location Data"),
            self.data_table,
            "---",
            pn.pane.Markdown("### 📈 Time Series Analysis"),
            self.chart_pane,
            sizing_mode='stretch_width'
        )
        
        # Complete layout
        layout = pn.template.MaterialTemplate(
            title="TEEHR Simulation Evaluation Dashboard",
            sidebar=[sidebar],
            main=[main_content],
            header_background='#2596be',
        )
        
        return layout

In [8]:
def create_locations_map(gdf):
    """
    Create Folium map with location markers using TEEHR visualization patterns.
    
    Args:
        gdf: GeoDataFrame with location data and geometries
        
    Returns:
        folium.Map: Interactive map centered on data extent
    """
    if gdf is None or len(gdf) == 0:
        # Default center if no data (geographic center of CONUS for TEEHR)
        center_lat, center_lon = 39.8283, -98.5795
    else:
        # Calculate center from GeoDataFrame bounds for optimal view
        # This follows TEEHR patterns for efficient geospatial processing
        bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
        center_lon = (bounds[0] + bounds[2]) / 2  # (minx + maxx) / 2
        center_lat = (bounds[1] + bounds[3]) / 2  # (miny + maxy) / 2
    
    # Create base map with multiple tile layers for better visualization
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=4,
        tiles=None  # We'll add custom layers
    )

    # Add multiple tile layers following TEEHR dashboard patterns
    folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m)
    # folium.TileLayer('CartoDB positron', name='CartoDB Light').add_to(m)
    # folium.TileLayer('CartoDB dark_matter', name='CartoDB Dark').add_to(m)
    
    # Add markers for each location with data quality color coding
    for idx, row in gdf.iterrows():
        geom = row.geometry
        if geom and not geom.is_empty:
            # Color code by data quality bucket following TEEHR patterns
            # color_map = {
            #     'high': 'green',    # High quality: >10k records
            #     'medium': 'orange', # Medium quality: 1k-10k records
            #     'low': 'red'        # Low quality: <1k records
            # }
            # color = color_map.get(getattr(row, 'record_count_bucket', 'low'), 'blue')
            color = "blue"
            
            # Create rich popup with TEEHR metadata
            popup_content = f"""
            <div style='min-width: 200px'>
                <p>{row["name"]}</br>{row["primary_location_id"]}</p>
                <p><b>Records:</b> {int(row["count"]):.0f}</p>
                <p><b>Coordinates:</b> {geom.y:.4f}°, {geom.x:.4f}°</p>
                <button onclick="window.parent.postMessage({{type: 'location_click', location_id: '{row['primary_location_id']}'}}, '*')">
                    View Time Series
                </button>
            </div>
            """
            
            folium.CircleMarker(
                location=[geom.y, geom.x],
                popup=folium.Popup(popup_content, max_width=300),
                tooltip=f"{row["primary_location_id"]}:{row["name"]}",
                radius=8,
                color='white',
                weight=2,
                fillColor=color,
                fillOpacity=0.8
            ).add_to(m)
    
    # Add layer control for tile selection
    folium.LayerControl().add_to(m)
    
    return m

In [10]:
class TEEHRDashboard:
    """
    Main dashboard class following TEEHR architecture patterns.
    
    Implements cloud-native hydrologic evaluation dashboard with:
    - ECS Trino integration for scalable data access
    - Interactive geospatial visualization
    - Time series analysis and comparison
    - Data quality assessment and visualization
    """
    
    def __init__(self):
        """Initialize dashboard components and state."""
        self.locations_gdf = None
        self.selected_location_id = None
        self.selected_location_data = None
        
        # Initialize Panel components
        self.setup_components()
        self.load_metrics_data()
    
    def setup_components(self):
        """Setup Panel dashboard components following TEEHR UI patterns."""
        
        # Header and configuration
        self.title = pn.pane.Markdown("""
        # 🗺️ TEEHR Simulation Evaluation Dashboard
        """)
        
        # Configuration panel
        self.config_info = pn.pane.Markdown(f"""
        ### 🔧 Database Connection Information
        ```
        Host: {TRINO_HOST}
        Port: {TRINO_PORT}
        User: {TRINO_USER}
        Catalog: {TRINO_CATALOG}
        Schema: {TRINO_SCHEMA}
        ```
        """)
        
        # Status indicator
        self.status_pane = pn.pane.Markdown("🔄 **Loading connection status...**")
        
        # Data refresh button
        self.refresh_button = pn.widgets.Button(
            name="🔄 Refresh Data", 
            button_type="primary",
            width=150
        )
        self.refresh_button.on_click(self.refresh_data)
        
        # Map component
        self.map_pane = pn.pane.HTML("Loading map...", height=600)
        
        # Summary statistics
        self.stats_pane = pn.pane.Markdown("Loading statistics...")
        
        # Location selection
        self.location_selector = pn.widgets.Select(
            name="📍 Select Location",
            options=[],
            width=300
        )
        self.location_selector.param.watch(self.on_location_change, 'value')
        
        # Date range selectors
        # today = date.today()
        # default_start = today - timedelta(days=30)
        
        # self.start_date = pn.widgets.DatePicker(
        #     name="📅 Start Date",
        #     value=default_start,
        #     end=today,
        #     width=150
        # )
        
        # self.end_date = pn.widgets.DatePicker(
        #     name="📅 End Date", 
        #     value=today,
        #     start=default_start,
        #     end=today,
        #     width=150
        # )
        
        # Time series chart
        self.chart_pane = pn.pane.Plotly(height=400)
        
        # Location details
        self.location_details = pn.pane.Markdown("Select a location to view details")
        
        # Data table
        self.data_table = pn.widgets.Tabulator(pagination='remote', page_size=20, height=300)
    
    def load_metrics_data(self):
        """Load locations data from Trino following TEEHR patterns."""
        self.status_pane.object = "🔄 **Loading data from ECS Trino...**"
        
        # Test connection first
        conn, conn_error = get_trino_connection()
        if conn_error:
            self.status_pane.object = f"❌ **Connection Failed**: {conn_error}"
            return
        
        # Load locations data
        gdf, error = get_locations_data()
        if error:
            self.status_pane.object = f"❌ **Data Loading Failed**: {error}"
            return
        
        self.locations_gdf = gdf
        self.update_components()
        self.status_pane.object = f"✅ **Connected**: Loaded {len(gdf)} locations"
    
    def update_components(self):
        """Update dashboard components with loaded data."""
        if self.locations_gdf is None:
            return
        
        # Update summary statistics
        gdf = self.locations_gdf
        bounds = gdf.total_bounds
        lat_range = bounds[3] - bounds[1]
        lon_range = bounds[2] - bounds[0]
        
        self.stats_pane.object = f"""
        ### 📊 Data Summary
        - **Total Locations**: {len(gdf):,}
        - **Valid Geometries**: {len(gdf[~gdf.geometry.is_empty]):,}
        - **Geographic Span**: {lat_range:.1f}° × {lon_range:.1f}°
        """
        
        # Update location selector
        location_options = list(gdf["primary_location_id"])
        self.location_selector.options = location_options
        
        # Update map
        locations_map = create_locations_map(gdf)
        self.map_pane.object = locations_map._repr_html_()
        
        # Update data table
        display_df = pd.DataFrame(gdf)
        display_df = display_df.drop(columns=["geometry"]).copy()
        # display_df['latitude'] = gdf.geometry.y.round(6)
        # display_df['longitude'] = gdf.geometry.x.round(6)
        self.data_table.value = display_df
    
    def refresh_data(self, event=None):
        """Refresh data from Trino."""
        self.load_metrics_data()
    
    def on_location_change(self, event):
        """Handle location selection change."""
        if not event.new:
            return
        
        self.selected_location_id = event.new
        filtered = self.locations_gdf[self.locations_gdf.primary_location_id == event.new]
        if filtered.empty:
            self.location_details.object = f"⚠️ No data found for location: {event.new}"
            return
        selected_row = filtered.iloc[0]
        
        # Update location details
        geom = selected_row.geometry
        self.location_details.object = f"""
        ### 📍 Selected Location: {selected_row.primary_location_id}
        - **Name**: {selected_row.name}
        - **Coordinates**: {geom.y:.4f}°N, {geom.x:.4f}°W
        - **Records**: {selected_row.count}
        """

        # Set Selector widget value
        self.location_selector.value = selected_row.primary_location_id
        
        # Load time series data
        self.load_timeseries()
    
    def load_timeseries(self):
        """Load time series data for selected location."""
        if not self.selected_location_id:
            return
        
        # Get date range
        # start_date = self.start_date.value
        # end_date = self.end_date.value
        
        # Load data
        primary_df, error = get_primary_timeseries(self.selected_location_id)
        
        if error:
            self.chart_pane.object = go.Figure().add_annotation(
                text=f"No time series data available: {error}",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            return

        secondary_df, error = get_secondary_timeseries(self.selected_location_id)

        if error:
            self.chart_pane.object = go.Figure().add_annotation(
                text=f"No time series data available: {error}",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False
            )
            return
            
        # Create chart
        chart = create_timeseries_chart(primary_df, secondary_df, self.selected_location_id)
        self.chart_pane.object = chart
    
    def create_layout(self):
        """Create dashboard layout following TEEHR UI patterns."""
        
        # Sidebar with configuration and controls
        sidebar = pn.Column(
            self.config_info,
            self.status_pane,
            self.refresh_button,
            "---",
            self.stats_pane,
            "---",
            self.location_selector,
            self.location_details,
            # pn.Row(self.start_date, self.end_date),
            width=350,
            scroll=True
        )
        
        # Main content area
        main_content = pn.Column(
            self.title,
            pn.pane.Markdown("🚀 **Using ECS-hosted Trino** to query TEEHR locations from Iceberg warehouse"),
            "---",
            pn.pane.Markdown("### 📍 Interactive Location Map"),
            self.map_pane,
            "---",
            pn.pane.Markdown("### 📋 Location Data"),
            self.data_table,
            "---",
            pn.pane.Markdown("### 📈 Time Series Analysis"),
            self.chart_pane,
            sizing_mode='stretch_width'
        )
        
        # Complete layout
        layout = pn.template.MaterialTemplate(
            title="TEEHR Locations Dashboard",
            sidebar=[sidebar],
            main=[main_content],
            header_background='#2596be',
        )
        
        return layout